In [1]:
import networkx as nx
import sys
import os
from termcolor import colored
from networkx.algorithms.community.label_propagation import *
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
CLASSES_PATH = os.path.dirname(os.path.abspath('../../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos
from Classes.SIR_Diffusion_Model_Class import SIR_Diffusion_Model
from Classes.Get_Past_Results_Class import Get_Past_Results
from Classes.Network_Infos_Writer_Class import Network_Infos_Writer
from Classes.K_Shell_Calculate_Class import K_Shell_Calculate


In [3]:
seed_size_list = [# [1, 4, 8, 12, 16, 20, 24, 28],
                  # [1, 4, 8, 12, 16, 20, 24, 28]
                  # [1, 3, 6, 9, 12, 15, 17, 19],
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 3, 6, 9, 12, 15, 17, 19]
                  # [1, 2, 4, 6, 8, 10, 12, 14],
                  # [1, 2, 3, 4, 5, 6, 8, 10],
                  # [1, 2, 3, 4, 5],
                  # [1, 2, 3, 4, 5]
                  ]
datasets_list = [# '01 - Sanremo2016_final (55897 Node and 3 Layer)/Sanremo2016_final.edgeslist',
                 # '02 - C_H_N (45583 Node and 3 Layer)/C_H_N.edgeslist'
                 # '03 - higgs-reply_network (38918 Node and 20 Layer)/higgs-reply_network.edgeslist'
                 # '04 - 2 AS (33752 Node and 7 Layer)/2 AS.edgeslist'
                 # '05 - friendfeed_ita (21006 Node and 3 Layer)/friendfeed_ita.edgeslist',
                 # '06 - Figs S1-S5 (10000 Node and 5 Layer)/Figs S1-S5.edgeslist'
                 # '07 - fftwyt (6407 Node and 3 Layer)/fftwyt.edgeslist',
                 # '08 - Irvine (1663 Node and 51 Layer)/Irvine.edgeslist',
                 # '09 - Wainwright (217 Node and 36 Layer)/Wainwright.edges',
                 # '10 - realitycommons (84 Node and 5 Layer)/Relationshipsfromsurveys.edgelist'
                
                ]

root_file_path = 'D:/Masters thesis/Methods For Compare/Datasets/' 



color_list = ["light_red", "light_green", "light_yellow",
                    "light_blue","light_magenta", "light_cyan",
                    "blue", "red", "white", "green", "yellow",
                        "magenta", "cyan", ]
tqdm_color_list = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'white']


files_handler_obj = Files_Handler()
SIR_diffusion_model_obj = SIR_Diffusion_Model()
read_network_infos_obj = Resd_Network_Infos()


source_code_path = str(os.getcwd())
source_code_path = source_code_path.replace("\\", "/")

In [4]:
def draw_graphs_colored_by_attribute(graphs_of_network:list[nx.Graph], attribute:str):
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import figure
    # create number for each group to allow use of colormap
    from itertools import count
    # get unique groups
    for j, graph in enumerate(graphs_of_network):
        if len(graph) > 0:
            groups = set(nx.get_node_attributes(graph, attribute).values())
            mapping = dict(zip(sorted(groups),count()))
            nodes = graph.nodes()
            colors = [mapping[graph.nodes[n][attribute]] for n in nodes]

            # drawing nodes and edges separately so we can capture collection for colobar
            pos = nx.spring_layout(graph)
            ec = nx.draw_networkx_edges(graph, pos, alpha=0.3)
            labels = nx.draw_networkx_labels(graph, pos=pos)
            nc = nx.draw_networkx_nodes(graph, pos, label=labels, nodelist=nodes, node_color=colors, cmap=plt.cm.jet)
            plt.colorbar(nc)
            plt.axis('off')
            plt.show()

def get_quota(k, n, v):
    quota = k * (n/v)
    return quota



In [5]:
for i, path in enumerate(datasets_list):
    file_path = root_file_path + path
    for item in seed_size_list[i]:
        print(path, item)
        seed_set_size = item
#--------------------------------------------------------------------------------------------------------------------------
        if file_path is None or file_path == '':
            sys.exit("File Selection Canceled !")
        file_info = files_handler_obj.get_file_path_info(file_path)
        network_name = file_info['name']
        network_type = file_info['type']
        network_path = file_info['path']
        if network_name == "":
            sys.exit("Dont Network Selection!")
        file_info
#--------------------------------------------------------------------------------------------------------------------------
        get_past_results_obj = Get_Past_Results(file_info['path'], file_info['name'])
        network_infos_writer_obj = Network_Infos_Writer(file_info['path'], file_info['name'])
#--------------------------------------------------------------------------------------------------------------------------
        (
            network_layers_info,
            network_layers_nodes,
            entra_layer_edges,
            entra_layer_edges_features,
            inter_layer_edge,
        ) = read_network_infos_obj.read_nodeFrom_layerFrom_nodeTo_layerTo(
            network_path, network_name, network_type
        )
#--------------------------------------------------------------------------------------------------------------------------
        print(file_info['name'], '\n')
        sum_of_layers_nodes = 0
        network_layers_count = len(network_layers_info)
        graphs_of_network = [None] * network_layers_count
        network_entier_edges = ""
        layers_nodes_infect_scale = []

        i = 0
        j = 0
        while i < network_layers_count:
            graphs_of_network[i] = nx.Graph()
            network_layers_nodes[i] = list(set(network_layers_nodes[i]))
            layers_nodes_infect_scale.append({})
            if len(network_layers_nodes[i]) > 0:
                # graphs_of_network[i].add_nodes_from(network_layers_nodes[i])
                graphs_of_network[i].add_edges_from(entra_layer_edges[i])
                nx.set_node_attributes(graphs_of_network[i], None, 'community')

                graphs_of_network[i].graph["id"] = i
                graphs_of_network[i].graph["node_num"] = graphs_of_network[i].number_of_nodes()
                graphs_of_network[i].graph["edge_num"] = graphs_of_network[i].number_of_edges()

                print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                            str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
                # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
            i += 1
            j += 1
            if j >= len(color_list):
                j = 0

        network_entier_nodes_list = []
        for item in network_layers_nodes:
            network_entier_nodes_list += item
            sum_of_layers_nodes += len(item)

        network_entier_nodes_list = list(set(network_entier_nodes_list))
        network_entier_nodes_count = len(network_entier_nodes_list)
        print()
        print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))
        k_shell_path = files_handler_obj.make_dir(file_info['path'], 'k_shell')
#--------------------------------------------------------------------------------------------------------------------------
        LPA_communities, LPA_communities_load_flag = get_past_results_obj.load_results_from_file(k_shell_path, 'LPA_communities')
        if LPA_communities_load_flag == False:
            LPA_communities = []
            for j, graph in enumerate(graphs_of_network):
                LPA_communities.append([])
                if graph.number_of_edges() > 0:
                    communities = label_propagation_communities(graph)
                    for i, item in enumerate(communities):
                        LPA_communities[j].append(list(item))
                        for node in item:
                            graph.nodes[node]["community"] = i
        else:
            for j, communities in enumerate(LPA_communities):
                if graphs_of_network[j].number_of_edges() > 0:
                    for i, item in enumerate(communities):
                        for node in item:
                            graphs_of_network[j].nodes[node]["community"] = i
#--------------------------------------------------------------------------------------------------------------------------
        if LPA_communities_load_flag == False:
            network_infos_writer_obj.write_results_in_file(k_shell_path, 'LPA_communities', LPA_communities)
#--------------------------------------------------------------------------------------------------------------------------
        community_counter = 0
        quota = {}
        communities_dict = {}
        sorted_communities_k_shell = {}
        for j, graph in enumerate(graphs_of_network):
            quota[j] = {}
            if graph.number_of_nodes() > 0:
                sorted_communities_k_shell[j] = {}
                communities = nx.get_node_attributes(graph, 'community')
                communities_dict[j] = {}
                for kk, v in communities.items():
                    if v not in communities_dict[j]:
                        communities_dict[j][v] = [kk]
                    else:
                        communities_dict[j][v].append(kk)
                K_Shell_Calculate_Object = K_Shell_Calculate(graph)
                for community, community_nodes in communities_dict[j].items():
                    community_counter += 1
                    community_graph = graph.subgraph(community_nodes)
                    community_graph = K_Shell_Calculate_Object.get_subgraph_k_shell_info(nx.Graph(community_graph))
                    quota[j][community] = get_quota(seed_set_size, len(community_nodes), sum_of_layers_nodes)


                    communities_k_shell = nx.get_node_attributes(community_graph, 'k_shell')
                    k_shell_grouping = {}
                    for kk, vv in communities_k_shell.items():
                        vote_power = community_graph.nodes[kk]['vote_power']
                        if vv not in k_shell_grouping:
                            k_shell_grouping[vv] = [(kk, vote_power)]
                        else:
                            k_shell_grouping[vv].append((kk, vote_power))
                    
                    k_shell_grouping = dict(sorted(k_shell_grouping.items(), reverse=True))
                    for kk in k_shell_grouping.keys():
                        k_shell_grouping[kk] = sorted(k_shell_grouping[kk], key=lambda item: item[1], reverse=True)
                    sorted_communities_k_shell[j][community] = k_shell_grouping
#--------------------------------------------------------------------------------------------------------------------------
        draw_status = False
        if draw_status:
            draw_graphs_colored_by_attribute(graphs_of_network, 'community')
#--------------------------------------------------------------------------------------------------------------------------
        sorted_quota = []
        for j in range(community_counter):
            max_quota_value = -1
            max_quota_cummunity = None
            max_quota_layer = None
            for i, _ in enumerate(quota.items()):
                if len(quota[i].keys()) > 0:
                    for kk, vv in quota[i].items():
                        if  vv > max_quota_value :
                            max_quota_value = vv
                            max_quota_cummunity = kk
                            max_quota_layer = i
            del quota[max_quota_layer][max_quota_cummunity]
            sorted_quota.append((max_quota_layer, max_quota_cummunity, max_quota_value))
                            
#--------------------------------------------------------------------------------------------------------------------------
        seed_set = []
        print('Seed set size: ', seed_set_size)
        for j, item in enumerate(sorted_quota):
            max_k_shell_number = max(sorted_communities_k_shell[item[0]][item[1]].keys())
            if item[2] < len(sorted_communities_k_shell[item[0]][item[1]][max_k_shell_number]):
                i = 0
                c = 0
                while i < item[2]:
                    candide = sorted_communities_k_shell[item[0]][item[1]][max_k_shell_number][c][0]
                    if not(candide in seed_set):
                        seed_set.append(candide)
                        i += 1
                        if len(seed_set) >= seed_set_size:
                            break
                    c += 1
            else:
                k_shell_number = sorted(sorted_communities_k_shell[item[0]][item[1]].keys(), reverse=True)
                i = 0
                for shell in k_shell_number:
                    c = 0
                    while i < item[2]:
                        candide = sorted_communities_k_shell[item[0]][item[1]][shell][c][0]
                        if not(candide in seed_set):
                            seed_set.append()
                            i += 1
                            if len(seed_set) >= seed_set_size:
                                break
                        c += 1
                    if len(seed_set) >= seed_set_size:
                        break
            if len(seed_set) >= seed_set_size:
                break

        print('seedset: ', seed_set)
#--------------------------------------------------------------------------------------------------------------------------
        print(f"Network name: {network_name}")
        print(f"Seed set size: {seed_set_size}")
        beta = 0.01
        landa = 0.7
        epoch = 10000

        infection = SIR_diffusion_model_obj.synchronous_SIR_multilayer_with_seed_set_model(graphs_of_network,
                                                                                        seed_set, beta,
                                                                                        landa, epoch,
                                                                                        network_entier_nodes_list)

        sir_results_infos = {}
        sir_results_infos['infection'] = infection
        sir_results_infos['percentage'] = infection / network_entier_nodes_count
        sir_results_infos['seed_set'] = seed_set

        network_infos_writer_obj.write_results_in_file(k_shell_path,
                                                    f'infection k={seed_set_size} beta={beta} landa{landa} epoch{epoch}',
                                                    sir_results_infos)
        print(f"Network entier nodes count: {network_entier_nodes_count}")
        print(f"Infected nodes count: {infection}")
        print(f"Percentage of infection: {sir_results_infos['percentage']}")
#--------------------------------------------------------------------------------------------------------------------------
        clear_output(wait=False)
#--------------------------------------------------------------------------------------------------------------------------
print('Finished!')

Finished!
